In [1]:
import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

data = {'getAPIKeyForUser': {'catalog': 'tecdocsw',
                             'username': '186299u15',
                             'password': 'APMTeDoW14'}}

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.post('https://webservice.tecalliance.services/webcat30/v1/services/WebCat30WS.jsonEndpoint',
                             data=json.dumps(data),
                             headers=get_header(),
                             #proxies=get_proxy(),
                             timeout=(10, 10))

        if resp.status_code == 200:
            json_ = resp.json()

            if json_['status'] == 200:
                key = json_['apiKey']
                break
    except KeyboardInterrupt:
        break
    except:
        continue

key

1


'2BeBXg69osbuG45Rjtv1wLEnHL3nuT3WTPto2C3dW65fAmyAFsoa'

In [2]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [3]:
input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['No'].tolist()

总数量：2


[1, 2]

In [4]:
a = 0

a

0

In [5]:
crawler_status = 'error'

crawler_status

'error'

In [6]:
url = input_.loc[a, 'Url']

dict_param = {}
list_param = url.split('#@')[0].split('?')[1].split('&') if '?' in url else []
for param in list_param:
    dict_param[param.split('=')[0]] = param.split('=')[1]

data = {'getArticles': {'articleCountry': 'DE',
                        'articleStatusIds': [],
                        'criteriaFilters': [],
                        'dataSupplierIds': [int(dict_param['brands'])],
                        'filterQueries':  ['(dataSupplierId NOT IN (4978,4982))'],
                        'genericArticleIds': [int(dict_param['groups'])],
                        'includeAccessoryArticles': False,
                        'includeAll': False,
                        'includeArticleCriteria': True,
                        'includeArticleLogisticsCriteria': True,
                        'includeArticleStatusFacets': True,
                        'includeArticleText': True,
                        'includeComparableNumbers': True,
                        'includeCriteriaFacets': True,
                        'includeDataSupplierFacets': True,
                        'includeGTINs': True,
                        'includeGenericArticleFacets': True,
                        'includePartsListArticles': False,
                        'includeImages': True,
                        'includeLinkages': True,
                        'includeLinks': True,
                        'includeMisc': True,
                        'includeOEMNumbers': True,
                        'includePDFs': True,
                        'includePartsListArticles': False,
                        'includePrices': False,
                        'includeReplacedByArticles': True,
                        'includeReplacesArticles': True,
                        'includeTradeNumbers': True,
                        'lang': 'en',
                        'linkagesPerPage': 100,
                        'page': 1,
                        'perPage': 1_000,
                        'provider': 40_000,
                        'sort': [{'field': 'articleNumber', 'direction': 'asc'}]}}

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                             data=json.dumps(data),
                             headers=get_header(**{'x-api-key': key}),
                             #proxies=get_proxy(),
                             timeout=(10, 10))

        if resp.status_code == 200:
            json_ = resp.json()

            if json_['status'] == 401 and json_['statusText'].strip() == 'Access not allowed':
                while True:
                    try:
                        data = {'getAPIKeyForUser': {'catalog': 'tecdocsw',
                                                     'username': '186299u15',
                                                     'password': 'APMTeDoW14'}}
                        
                        resp = requests.post('https://webservice.tecalliance.services/webcat30/v1/services/WebCat30WS.jsonEndpoint',
                                             data=json.dumps(data),
                                             headers=get_header(),
                                             #proxies=get_proxy(),
                                             timeout=(10, 10))
                
                        if resp.status_code == 200:
                            json_ = resp.json()
                
                            if json_['status'] == 200:
                                key = json_['apiKey']
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue
            elif json_['status'] == 200:
                break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1


<Response [200]>

In [7]:
count = json_['totalMatchingArticles']

count

1900

In [8]:
count <= 0

False

In [9]:
import math

df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'Url': f'''https://web.tecalliance.net/tecdocsw/en/parts/search?brands={int(dict_param['brands'])}&groups={int(dict_param['groups'])}''',
                         'SKU Count': count,
                         'Page': math.ceil(count / 1_000)}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Url,SKU Count,Page
0,1,https://web.tecalliance.net/tecdocsw/en/parts/...,1900,2


In [10]:
crawler_status = 'ok'

crawler_status

'ok'

In [11]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [12]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_page.xlsx', index=False)

output_correct

,No,Url,SKU Count,Page
0,1,https://web.tecalliance.net/tecdocsw/en/parts/...,1900,2


In [13]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_page_error.xlsx', index=False)

output_error

""
